이미지를 증식해서 넣어봅시다!!

In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator


import pandas as pd 
import os
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten
from keras.utils import to_categorical

from keras.models import Sequential,load_model

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
np.random.seed(1)

In [7]:
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        './images2/train',
        target_size = (300,300),
        batch_size=50,
#         color_mode ='rgb',
        classes = ['food','interior', 'exterior'],
        shuffle =True,
        seed = 42,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        './images2/test',
#         classes =[],
        target_size=(300,300),
        batch_size=10,
#         color_mode ='rgb',
        shuffle =True,
        seed = 42,
        class_mode='categorical')


Found 36000 images belonging to 3 classes.
Found 9000 images belonging to 3 classes.


In [14]:
iterations = 150

train_plus = train_datagen.flow_from_directory(
    './images2/train',
    target_size = (300,300),
    batch_size = 10,
    class_mode = 'categorical')

for i in enumerate(range(iterations)):
    img, label = train_plus.next()
#     n_img = len(label)
    
#     base = cv2.cvtColor(img[0], cv2.COLOR_RGB2BGR)  # keras는 RGB, openCV는 BGR이라 변경함
#     for idx in range(n_img - 1):
#         img2 = cv2.cvtColor(img[idx + 1], cv2.COLOR_RGB2BGR)
#         base = np.hstack((base, img2))
#     images.append(base)
 
# img = images[0]
# for idx in range(len(images) - 1):
#     img = np.vstack((img, images[idx + 1]))
# cv2.imshow('result', img)

Found 36000 images belonging to 3 classes.


In [20]:
model = Sequential()
model.add(Conv2D(64, kernel_size=(3,3),
                 activation='selu',
                 input_shape=(300,300,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32, (3,3), activation='selu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Flatten())
# model.add(Dense(32, activation='selu'))
# model.add(Dense(16, activation='relu'))
model.add(Dense(3, activation='softmax'))

# 3. 모델 학습과정 설정 및 요약
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 4. 모델 학습시키기
model.fit_generator(
        train_generator,
        steps_per_epoch=5,
        epochs=10,
        validation_data=test_generator,
        validation_steps=5)

model.save('model-11')

# 5. 모델 평가하기
print("-- Evaluate --")
scores = model.evaluate_generator(test_generator, steps=5)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 6. 모델 사용하기
print("-- Predict --")
output = model.predict_generator(test_generator, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_generator.class_indices)
print(output)

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 298, 298, 64)      1792      
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 149, 149, 64)      0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 147, 147, 32)      18464     
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 73, 73, 32)        0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 170528)            0         
_________________________________________________________________
dense_12 (Dense)             (None, 3)                 511587    
Total params: 531,843
Trainable params: 531,843
Non-trainable params: 0
_______________________________________________

In [22]:
def test_load_data():
    train_images = []       
    train_labels = []
    shape = (300,300)  
    train_path = './image_test'

    for classes in ['f','i','e']:
        for filename in os.listdir('./image_test'):
            if classes == filename[0]:
                img = cv2.imread(os.path.join(train_path,filename))
                train_labels.append(filename[2:0:-1])

                # Resize all images to a specific shape
                #img = cv2.resize(img,shape)

                train_images.append(img)
            else:
                pass

    # Converting labels into One Hot encoded sparse matrix

    # 음식, 실내 ,실외 순으로 들어가기 위해 index 처리
    # train_labels = sorted(train_labels)
    # print(train_labels)

    train_labels = pd.get_dummies((train_labels)).values
    # print(train_labels)
    # Converting train_images to array
    train_images = np.array(train_images)

    # Splitting Training data into train and validation dataset

    train_images ,train_labels= shuffle(train_images ,train_labels, random_state = 42) 

    return train_images, train_labels

def predict_image_sample(model,X_test, y_test, test_id= -1 ):
    if test_id <0 :
        from random import randrange
        test_sample_id = randrange(10000)
    else:
        test_sample_id = test_id
    
    test_image =X_test[test_sample_id]
#     plt.imshow(test_images[123])

    
    test_image = test_image.reshape(1,300,300,3)
    
    y_actual = y_test[test_sample_id].tolist().index(1)
    print('y_actual number = ', y_actual)
    
    y_pred = model.predict(test_image)
    print("y_pred = ", y_pred)
    
    y_pred_num = np.argmax(y_pred, axis = 1)[0]
    print('y_pred number = ', y_pred_num)
    return y_actual , y_pred_num


In [26]:
if __name__ == '__main__' :
    n = 0
    model = load_model('model-10')
    X_test, y_test =test_load_data()
    for i in range(20):
        y_actual , y_pred_num = predict_image_sample(model,X_test, y_test,test_id = i)
        
        if y_actual == y_pred_num:
            n += 1 
        else : pass
print('전체 횟수 :', i,' '*10,'예측 성공 횟수',n,'\n'*2 ,'정확도 = ', (n/i)*100)

y_actual number =  1
y_pred =  [[0.000 0.000 1.000]]
y_pred number =  2
y_actual number =  1
y_pred =  [[0.000 0.000 1.000]]
y_pred number =  2
y_actual number =  1
y_pred =  [[0.000 0.000 1.000]]
y_pred number =  2
y_actual number =  1
y_pred =  [[0.000 0.000 1.000]]
y_pred number =  2
y_actual number =  1
y_pred =  [[0.000 0.000 1.000]]
y_pred number =  2
y_actual number =  1
y_pred =  [[0.000 0.000 1.000]]
y_pred number =  2
y_actual number =  0
y_pred =  [[0.000 0.000 1.000]]
y_pred number =  2
y_actual number =  0
y_pred =  [[0.000 0.000 1.000]]
y_pred number =  2
y_actual number =  0
y_pred =  [[0.000 0.000 1.000]]
y_pred number =  2
y_actual number =  1
y_pred =  [[0.000 0.000 1.000]]
y_pred number =  2
y_actual number =  0
y_pred =  [[0.000 0.000 1.000]]
y_pred number =  2
y_actual number =  0
y_pred =  [[0.000 0.000 1.000]]
y_pred number =  2
y_actual number =  0
y_pred =  [[0.000 0.000 1.000]]
y_pred number =  2
y_actual number =  1
y_pred =  [[0.000 0.000 1.000]]
y_pred numb